### Data Cleaning 

### TSLA

In [ ]:
from pathlib import Path
import pandas as pd
from typing import Iterable

# Resolve project root (works from repo root or notebooks/)
cwd = Path.cwd().resolve()
PROJECT_ROOT = cwd.parent if cwd.name == 'notebooks' else cwd

price_path = PROJECT_ROOT / 'data' / 'interim' / 'NVDA_price_full.csv'
sent_path  = PROJECT_ROOT / 'data' / 'interim' / 'sentiment_scoring' / 'nvda_daily_sentiment_score.csv'
spike_path = PROJECT_ROOT / 'data' / 'interim' / 'activitiy_recognition' / 'nvda_spike_data.csv'  # folder name as-is

out_path = PROJECT_ROOT / 'data' / 'processed' / 'nvda_price_sentiment_spike_merged.csv'
out_path.parent.mkdir(parents=True, exist_ok=True)

def coerce_date_col(df: pd.DataFrame, candidates: Iterable[str]) -> pd.DataFrame:
    df = df.copy()
    # candidate list + heuristics (any column containing date/time hints)
    heuristics = [c for c in df.columns if any(k in c.lower() for k in ('date','day','time','timestamp','created'))]
    for col in list(dict.fromkeys([*candidates, *heuristics])):  # preserve order, dedupe
        if col not in df.columns:
            continue
        s = df[col]
        dt = pd.to_datetime(s, utc=True, errors='coerce')  # strings or datetimes
        # try epoch seconds/millis if needed
        if dt.isna().all() and pd.api.types.is_numeric_dtype(s):
            dt = pd.to_datetime(s, unit='s', utc=True, errors='coerce')
            if dt.isna().all():
                dt = pd.to_datetime(s, unit='ms', utc=True, errors='coerce')
        if dt.isna().all():
            continue
        # strip tz and keep date
        try:
            dt = dt.dt.tz_localize(None)
        except Exception:
            pass
        df['date'] = dt.dt.floor('D')
        return df
    raise ValueError(f'No date-like column found. columns={list(df.columns)}')

# 1) Price (anchor; keep all dates)
price = pd.read_csv(price_path)
price = coerce_date_col(price, candidates=('Date','date'))
price = price.sort_values('date').reset_index(drop=True)

# 2) Sentiment (daily)
sent = pd.read_csv(sent_path)
sent = coerce_date_col(sent, candidates=('date','Date'))
# normalize sentiment column name
if 'daily_sentiment' not in sent.columns:
    score_cols = [c for c in sent.columns if 'sentiment' in c.lower()]
    if not score_cols:
        raise ValueError(f'No sentiment score column found in {sent_path}')
    sent = sent.rename(columns={score_cols[0]: 'daily_sentiment'})
sent = sent[['date','daily_sentiment']].drop_duplicates('date')

# 3) Spike/activity
spike = pd.read_csv(spike_path)
spike = coerce_date_col(
    spike,
    candidates=('date','Date','datetime','impact_day','impact_date','impact_trading_day','created_utc','created_at','timestamp')
)

# keep only known/likely spike columns if present
spike_candidates = ['post_count','spike_presence','spike_intensity']
present_spike_cols = [c for c in spike_candidates if c in spike.columns]
spike = spike[['date', *present_spike_cols]].copy()

# if multiple rows per date exist, aggregate reasonably
if spike.duplicated('date').any():
    agg = {}
    for c in present_spike_cols:
        if 'count' in c:
            agg[c] = 'sum'
        elif 'presence' in c:
            agg[c] = 'sum'  # if presence is daily count/flag; adjust to 'max' if you prefer
        elif 'intensity' in c:
            agg[c] = 'max'
        else:
            agg[c] = 'first'
    spike = spike.groupby('date', as_index=False).agg(agg)

# 4) Merge (LEFT on price)
merged = price.merge(sent, on='date', how='left').merge(spike, on='date', how='left')

# Save
merged.to_csv(out_path, index=False)
print('rows:', len(merged))
print('saved:', out_path)
print('columns:', merged.columns.tolist())

In [ ]:
df = merged


#select date from 2025-02-03 to 2025-10-16
df= df[df['date'] >= '2025-02-03']
df = df[df['date'] <= '2025-07-17']


#how many daily sentiment are NaN?
print(df['daily_sentiment'].isna().sum())

In [ ]:
# sentiment
df['daily_sentiment'] = pd.to_numeric(df['daily_sentiment'], errors='coerce')
df['has_sentiment'] = df['daily_sentiment'].notna().astype(int)
s = df['daily_sentiment'].shift(1)
for w in (3,7,14):
    df[f'daily_sentiment_mean_{w}'] = s.rolling(w, min_periods=1).mean()
    df[f'daily_sentiment_std_{w}']  = s.rolling(w, min_periods=1).std()

# spike
for c in ('post_count','spike_presence','spike_intensity'):
    if c in df:
        df[c] = df[c].fillna(0)
df['has_spike'] = ((df.get('post_count',0)>0) | (df.get('spike_presence',0)>0) | (df.get('spike_intensity',0)>0)).astype(int)

# 입력 컬럼 NaN 제거(롤링/래그 등만)
input_cols = [c for c in df.columns if c.startswith('daily_sentiment_')] + \
             [c for c in ('post_count','spike_presence','spike_intensity') if c in df]
df[input_cols] = df[input_cols].fillna(method='ffill').fillna(0)


#drop open,high,low columns
# df = df.drop(columns=['Open', 'High', 'Low'])

# Standardize column names
df = df.rename(columns={"Close":"close","Volume":"volume"})
if "Date" in df.columns: df = df.drop(columns=["Date"])

# Sort and index
df = df.sort_values("date").reset_index(drop=True)
if "time_idx" not in df.columns:
    df["time_idx"] = range(len(df))
if "unique_id" not in df.columns:
    df["unique_id"] = "NVDA"

# Fill missing spike columns with 0 (meaning absence)
for c in ("post_count","spike_presence","spike_intensity"):
    if c in df.columns:
        df[c] = df[c].fillna(0)

# Sentiment rolling features should not contain NaN (allow only ffill/bfill right before model input)
sent_cols = [c for c in df.columns if c.startswith("daily_sentiment_")]
if sent_cols:
    df[sent_cols] = df[sent_cols].ffill().bfill()

df_to_csv = df.to_csv('data/processed/nvda_price_sentiment_spike_merged_20250203_20250717.csv', index=False)

In [ ]:
df

### NVDA

In [37]:
from pathlib import Path
import pandas as pd
import numpy as np

PROJECT_ROOT = Path('/Users/hwang-yejin/Desktop/Financial Time Series Forecasting with Deep Learning Models and Social Media Sentiment')

PRICE_CSV = PROJECT_ROOT / 'data' / 'processed' / 'TSLA_price_full.csv'
SPIKE_CSV = PROJECT_ROOT / 'data' / 'interim' / 'activitiy_recognition' / 'tsla_spike_data.csv'
SENTI_CSV = PROJECT_ROOT / 'data' / 'interim' / 'sentiment_scoring' / 'tsla_daily_sentiment_score.csv'
OUT_DIR   = PROJECT_ROOT / 'data' / 'processed'

def coerce_date_col(df: pd.DataFrame, candidates=('date','Date','datetime','timestamp','created','time','day')) -> pd.DataFrame:
  df = df.copy()
  # try explicit candidates first, then heuristic scan
  heuristics = [c for c in df.columns if any(k in c.lower() for k in ('date','day','time','timestamp','created'))]
  for col in list(candidates) + heuristics:
    if col not in df.columns:
      continue
    s = df[col]
    # try generic parse
    dt = pd.to_datetime(s, utc=True, errors='coerce')
    # try seconds epoch
    if dt.isna().all() and pd.api.types.is_numeric_dtype(s):
      dt = pd.to_datetime(s, unit='s', utc=True, errors='coerce')
      # try milliseconds epoch
      if dt.isna().all():
        dt = pd.to_datetime(s, unit='ms', utc=True, errors='coerce')
    if not dt.isna().all():
      df['date'] = dt.dt.tz_localize(None).dt.floor('D')
      return df
  raise ValueError('No date-like column found')

def group_by_date_mean_numeric(df: pd.DataFrame) -> pd.DataFrame:
  if 'date' not in df.columns:
    return df
  if not df['date'].is_monotonic_increasing or df.duplicated('date').any():
    num_cols = df.select_dtypes(include=[np.number, 'boolean']).columns.tolist()
    num_cols = [c for c in num_cols if c != 'date']
    agg = {c: 'mean' for c in num_cols}
    # for non-numeric, keep first
    for c in df.columns:
      if c not in agg and c != 'date':
        agg[c] = 'first'
    df = (df.groupby('date', as_index=False).agg(agg))
  return df

# 1) Load price and normalize date
price = pd.read_csv(PRICE_CSV)
# TSLA_price_full already has 'Date'; keep everything and add 'date'
price_date = pd.to_datetime(price['Date'], utc=True, errors='coerce').dt.tz_localize(None).dt.floor('D')
price = price.assign(date=price_date).sort_values('date').reset_index(drop=True)

# 2) Load spike, coerce date, collapse duplicates
spike = pd.read_csv(SPIKE_CSV)
spike = coerce_date_col(spike)
spike = group_by_date_mean_numeric(spike)

# 3) Load sentiment, coerce date, collapse duplicates
senti = pd.read_csv(SENTI_CSV)
senti = coerce_date_col(senti)
senti = group_by_date_mean_numeric(senti)

# Optional: add prefixes to avoid accidental name collisions (skip 'date')
def add_prefix(df, prefix):
  keep = ['date']
  cols = {c: (c if c in keep else f'{prefix}{c}') for c in df.columns}
  return df.rename(columns=cols)

spike = add_prefix(spike, 'spike_')

# 4) Left-join on price dates (anchor)
merged = (
  price.merge(senti, on='date', how='left', suffixes=('', '_dup'))
       .merge(spike, on='date', how='left', suffixes=('', '_dup2'))
       .sort_values('date')
       .reset_index(drop=True)
)

# Drop any accidental duplicate helper columns if created
dup_cols = [c for c in merged.columns if c.endswith('_dup') or c.endswith('_dup2')]
if dup_cols:
  merged = merged.drop(columns=dup_cols)


In [38]:
df= merged 

#select date from 2025-02-03 to 2025-10-16
df= df[df['date'] >= '2025-02-03']
df = df[df['date'] <= '2025-07-17']

df.rename(columns={'daily_sentiment_daily_sentiment':'daily_sentiment'}, inplace=True)

In [39]:
# df.drop(columns=['Open', 'High', 'Low'], inplace=True)
# df.rename(columns = {"Close":"close","Volume":"volume"}, inplace=True)

In [40]:
df

,Date,Open,High,Low,Close,Volume,last_earnings_date,days_since_earning,month,day_of_week,...,time_idx,unique_id,date,daily_sentiment,spike_impact_trading_day,spike_post_count,spike_smoothed,spike_spike_presence,spike_spike_intensity,spike_loess_upper
636,2025-02-03,386.679993,389.170013,374.359985,383.679993,93732100,2025-01-29 16:12:00-05:00,4.0,2,0,...,636,TSLA,2025-02-03,0.846911,2025-02-03,4.0,2.551097,0.0,0.0,5.519992
637,2025-02-04,382.630005,394.000000,381.399994,392.209991,57072200,2025-01-29 16:12:00-05:00,5.0,2,1,...,637,TSLA,2025-02-04,-0.666667,2025-02-04,1.0,2.248485,0.0,0.0,5.217380
638,2025-02-05,387.510010,388.390015,375.529999,378.170013,57223300,2025-01-29 16:12:00-05:00,6.0,2,2,...,638,TSLA,2025-02-05,0.868623,2025-02-05,1.0,1.983199,0.0,0.0,4.952094
639,2025-02-06,373.029999,375.399994,363.179993,374.320007,77918200,2025-01-29 16:12:00-05:00,7.0,2,3,...,639,TSLA,2025-02-06,0.000000,2025-02-06,1.0,1.852118,0.0,0.0,4.821012
640,2025-02-07,370.190002,380.549988,360.339996,361.619995,70298300,2025-01-29 16:12:00-05:00,8.0,2,4,...,640,TSLA,2025-02-07,0.887983,2025-02-07,2.0,1.876135,0.0,0.0,4.845030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,2025-07-11,307.890015,314.089996,305.649994,313.510010,79236400,2025-04-22 16:07:00-04:00,79.0,7,4,...,745,TSLA,2025-07-11,0.500000,2025-07-11,1.0,1.860793,0.0,0.0,4.829688
746,2025-07-14,317.730011,322.600006,312.670013,316.899994,78043400,2025-04-22 16:07:00-04:00,82.0,7,0,...,746,TSLA,2025-07-14,0.889710,2025-07-14,1.0,1.770486,0.0,0.0,4.739381
747,2025-07-15,319.679993,321.200012,310.500000,310.779999,77556300,2025-04-22 16:07:00-04:00,83.0,7,1,...,747,TSLA,2025-07-15,NaN,2025-07-15,1.0,2.259119,0.0,0.0,5.228013
748,2025-07-16,312.799988,323.500000,312.619995,321.670013,97284800,2025-04-22 16:07:00-04:00,84.0,7,2,...,748,TSLA,2025-07-16,0.891185,2025-07-16,2.0,2.876913,0.0,0.0,5.845808


In [41]:
#how many daily sentiment are NaN?
print(df['daily_sentiment'].isna().sum())

13


In [42]:
# sentiment
df['daily_sentiment'] = pd.to_numeric(df['daily_sentiment'], errors='coerce')

df['has_sentiment'] = df['daily_sentiment'].notna().astype(int)
df['has_sentiment'] = df['daily_sentiment'].notna().astype(int)


s = df['daily_sentiment'].shift(1)
for w in (3,7,14):
    df[f'daily_sentiment_mean_{w}'] = s.rolling(w, min_periods=1).mean()
    df[f'daily_sentiment_std_{w}']  = s.rolling(w, min_periods=1).std()

In [43]:

#spike
for c in ('post_count', 'spike_presence', 'spike_intensity'):
    if c in df:
        df[c] = df[c].fillna(0)
df['has_spike'] = (
    (df.get('post_count', pd.Series(0, index=df.index)) > 0) |
    (df.get('spike_presence', pd.Series(0, index=df.index)) > 0) |
    (df.get('spike_intensity', pd.Series(0, index=df.index)) > 0)
).astype(int)



# Sort and index
df = df.sort_values("date").reset_index(drop=True)
if "time_idx" not in df.columns:
    df["time_idx"] = range(len(df))
if "unique_id" not in df.columns:
    df["unique_id"] = "NVDA"

# Fill missing spike columns with 0 (meaning absence)
for c in ("post_count","spike_presence","spike_intensity"):
    if c in df.columns:
        df[c] = df[c].fillna(0)

# Sentiment rolling features should not contain NaN (allow only ffill/bfill right before model input)
sent_cols = [c for c in df.columns if c.startswith("daily_sentiment_")]
if sent_cols:
    df[sent_cols] = df[sent_cols].ffill().bfill()


In [44]:
# Remove NaN values from input columns (only rolling/lag features)
input_cols = [c for c in df.columns if c.startswith('daily_sentiment_')] + \
             [c for c in ('post_count','spike_presence','spike_intensity') if c in df]
df[input_cols] = df[input_cols].fillna(method='ffill').fillna(0)

/var/folders/pq/ywh3rrys6n9dnn8yfd90rznm0000gn/T/ipykernel_35998/4101972365.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[input_cols] = df[input_cols].fillna(method='ffill').fillna(0)


In [45]:
df

,Date,Open,High,Low,Close,Volume,last_earnings_date,days_since_earning,month,day_of_week,...,spike_spike_intensity,spike_loess_upper,has_sentiment,daily_sentiment_mean_3,daily_sentiment_std_3,daily_sentiment_mean_7,daily_sentiment_std_7,daily_sentiment_mean_14,daily_sentiment_std_14,has_spike
0,2025-02-03,386.679993,389.170013,374.359985,383.679993,93732100,2025-01-29 16:12:00-05:00,4.0,2,0,...,0.0,5.519992,1,0.846911,1.070261,0.846911,1.070261,0.846911,1.070261,0
1,2025-02-04,382.630005,394.000000,381.399994,392.209991,57072200,2025-01-29 16:12:00-05:00,5.0,2,1,...,0.0,5.217380,1,0.846911,1.070261,0.846911,1.070261,0.846911,1.070261,0
2,2025-02-05,387.510010,388.390015,375.529999,378.170013,57223300,2025-01-29 16:12:00-05:00,6.0,2,2,...,0.0,4.952094,1,0.090122,1.070261,0.090122,1.070261,0.090122,1.070261,0
3,2025-02-06,373.029999,375.399994,363.179993,374.320007,77918200,2025-01-29 16:12:00-05:00,7.0,2,3,...,0.0,4.821012,1,0.349622,0.880199,0.349622,0.880199,0.349622,0.880199,0
4,2025-02-07,370.190002,380.549988,360.339996,361.619995,70298300,2025-01-29 16:12:00-05:00,8.0,2,4,...,0.0,4.845030,1,0.067319,0.769856,0.262217,0.739635,0.262217,0.739635,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,2025-07-11,307.890015,314.089996,305.649994,313.510010,79236400,2025-04-22 16:07:00-04:00,79.0,7,4,...,0.0,4.829688,1,-0.033357,0.016115,0.368044,0.370627,0.468990,0.300142,0
110,2025-07-14,317.730011,322.600006,312.670013,316.899994,78043400,2025-04-22 16:07:00-04:00,82.0,7,0,...,0.0,4.739381,1,0.144428,0.308145,0.311235,0.303751,0.450591,0.289895,0
111,2025-07-15,319.679993,321.200012,310.500000,310.779999,77556300,2025-04-22 16:07:00-04:00,83.0,7,1,...,0.0,5.228013,0,0.455916,0.457432,0.421193,0.378631,0.482944,0.316531,0
112,2025-07-16,312.799988,323.500000,312.619995,321.670013,97284800,2025-04-22 16:07:00-04:00,84.0,7,2,...,0.0,5.845808,1,0.694855,0.275567,0.400608,0.419552,0.481821,0.331955,0


In [47]:
# 5) Save
OUT_DIR.mkdir(parents=True, exist_ok=True)
start, end = merged['date'].min().strftime('%Y%m%d'), merged['date'].max().strftime('%Y%m%d')
out_path = OUT_DIR / f'tsla_price_sentiment_spike_merged_{start}_{end}.csv'
df.to_csv(out_path, index=False)

print('Merged shape:', merged.shape)
print('Date range  :', merged["date"].min(), '→', merged["date"].max())
print('Saved to    :', out_path)
print('NaN counts (tail):')
print(merged.isna().sum().sort_values(ascending=False).head(15))

Merged shape: (791, 27)
Date range  : 2022-07-21 00:00:00 → 2025-09-15 00:00:00
Saved to    : /Users/hwang-yejin/Desktop/Financial Time Series Forecasting with Deep Learning Models and Social Media Sentiment/data/processed/tsla_price_sentiment_spike_merged_20220721_20250915.csv
NaN counts (tail):
spike_loess_upper           570
spike_spike_intensity       570
spike_spike_presence        570
spike_smoothed              570
spike_post_count            570
spike_impact_trading_day    570
daily_sentiment             559
rolling_volatility            0
date                          0
unique_id                     0
time_idx                      0
cumulative_return             0
return_1d                     0
Date                          0
Open                          0
dtype: int64
